This is based off B. Schwessinger's PST104E genome project

###### Assembltic was run with the following parameters###

* anchor of 8000 based on the TE size
* windows either being 10kp or 50kp



In [2]:
%matplotlib inline

In [3]:
from __future__ import print_function
import pandas as pd
from Bio import SeqIO
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from itertools import product


In [4]:
def return_contig_id(x):
    import re
    pattern = re.compile('_([0-9]+_?[0-9]*)')
    return pattern.search(x).groups()[0]

In [5]:
#define the input folder
#ASSEMBLETIC_FODLER = '/Users/havasquezgross/rust_project/Assembletics'
ASSEMBLETIC_FODLER = '/Users/havasquezgross/rust_project/nucmer_coverage/Assembletics'
FIGURE_PATH = '/Users/havasquezgross/rust_project/figures'

In [6]:
os.chdir(ASSEMBLETIC_FODLER)

In [7]:
#get the folders and the pcontigs fasta files
_8kbp_folders = [x for x in os.listdir(ASSEMBLETIC_FODLER) if x.endswith('_8kbp')]
pcontig_fa = [x for x in os.listdir(ASSEMBLETIC_FODLER) if x.endswith('.fa') and 'F' in x and 'h_ctgs' not in x]

In [8]:
len(_8kbp_folders) - len(pcontig_fa) #makes sense as this is the number of pwoh contigs plus whole assembly 

-84

In [9]:
#make dataframe for 10kb windows
_8kbp_folders.sort()
Assemblytics_summary_df_list_10kb = []
#Assemblytics_summary_df_list_50kb = []
variation_types = ['Insertion', 'Deletion', 'Tandem_expansion', 'Tandem_contraction', 'Repeat_expansion', 'Repeat_contraction']
for folder in _8kbp_folders:
    #get the summary file
    summary_file_10kb = [x for x in os.listdir(os.path.abspath(folder)) \
                    if x.endswith('10kp.Assemblytics_structural_variants.summary.csv')][0]
    #summary_file_50kb = [x for x in os.listdir(os.path.abspath(folder)) \
              #      if x.endswith('_50kp.Assemblytics_structural_variants.summary.csv')][0]
    file_name_10kb = ''
    file_name_10kb = os.path.join(ASSEMBLETIC_FODLER, folder, summary_file_10kb)
    #file_name_50kb = ''
    #file_name_50kb = os.path.join(base_folder, folder, summary_file_50kb)
    #now convert summary file to a df usable file
    with open(file_name_10kb) as s_handle:
        out_handle = open(file_name_10kb.replace('.csv', '.df'), 'w')
        variation_type = ''
        for line in s_handle:
            if line.strip() == '':
                continue
            if 'Total' in line:
                continue
            line = line.strip('\n')
            if line in variation_types:
                variation_type = line
                next
            else:
                line = variation_type + ','+line
                print(line, file=out_handle)
        out_handle.close()  
    #now read in the dataframe and reformat the indexes so it is useful later on
    #Multiindex column contig_id, numbers; Multiindex rows Type and size range
    contig = ''
    contig = return_contig_id(folder)
    #print('Reading in %s' % (contig))
    var_df = ''
    var_df = pd.read_csv(file_name_10kb.replace('.csv', '.df'), header=None, names=['Type', 'Size range', 'Count', 'Total bp'])
    if len(var_df) == 0: #some contigs might not have any variations called.
        continue
    index = pd.MultiIndex.from_tuples(list(zip(var_df['Type'],var_df['Size range'])), names=['Type', 'Size range'])
    var_df = var_df.set_index(index)
    var_df = var_df.iloc[:,2:] #subset the two remaining useful columns
    old_columns = list(var_df.columns)
    col_index = pd.MultiIndex.from_tuples(list(zip([contig, contig], old_columns)),names = ['contig_id','numbers'])
    var_df.columns = col_index
    Assemblytics_summary_df_list_10kb.append(var_df)
Assemblytics_summary_df_10kb = pd.concat(Assemblytics_summary_df_list_10kb, axis=1)

In [10]:
#make dataframe for 50kb windows
_8kbp_folders.sort()
Assemblytics_summary_df_list_50kb = []
variation_types = ['Insertion', 'Deletion', 'Tandem_expansion', 'Tandem_contraction', 'Repeat_expansion', 'Repeat_contraction']
for folder in _8kbp_folders:
    #get the summary file
    
    summary_file_50kb = [os.path.join(os.path.abspath(folder), x) for x in os.listdir(os.path.abspath(folder)) \
              if x.endswith('_50kp.Assemblytics_structural_variants.summary.csv')][0]
    file_name_50kb = ''
    file_name_50kb = os.path.join(ASSEMBLETIC_FODLER, folder, summary_file_50kb)
    #now convert summary file to a df usable file
    with open(summary_file_50kb) as s_handle:
        out_handle = open(file_name_50kb.replace('.csv', '.df'), 'w')
        variation_type = ''
        for line in s_handle:
            if line.strip() == '':
                continue
            if 'Total' in line:
                continue
            line = line.strip('\n')
            if line in variation_types:
                variation_type = line
                next
            else:
                line = variation_type + ','+line
                print(line, file=out_handle)
        out_handle.close()  
    #now read in the dataframe and reformat the indexes so it is useful later on
    #Multiindex column contig_id, numbers; Multiindex rows Type and size range
    contig = ''
    contig = return_contig_id(folder)
    #print('Reading in %s' % (contig))
    var_df = ''
    var_df = pd.read_csv(file_name_50kb.replace('.csv', '.df'), header=None, names=['Type', 'Size range', 'Count', 'Total bp'])
    if len(var_df) == 0: #some contigs might not have any variations called.
        continue
    index = pd.MultiIndex.from_tuples(list(zip(var_df['Type'],var_df['Size range'])), names=['Type', 'Size range'])
    var_df = var_df.set_index(index)
    var_df = var_df.iloc[:,2:] #subset the two remaining useful columns
    old_columns = list(var_df.columns)
    col_index = pd.MultiIndex.from_tuples(list(zip([contig, contig], old_columns)),names = ['contig_id','numbers'])
    var_df.columns = col_index
    Assemblytics_summary_df_list_50kb.append(var_df)
Assemblytics_summary_df_50kb = pd.concat(Assemblytics_summary_df_list_50kb, axis=1)

In [11]:
#save out the summary dataframes
Assemblytics_summary_df_50kb.to_csv(os.path.join(ASSEMBLETIC_FODLER, 'Assemblytics_summary_df_50kb.df'))
Assemblytics_summary_df_10kb.to_csv(os.path.join(ASSEMBLETIC_FODLER, 'Assemblytics_summary_df_10kb.df'))

In [12]:
Assemblytics_summary_df_10kb.unstack()

contig_id           000000                                                    \
numbers              Count                                 Total bp            
Size range         1-10 bp 10-50 bp 50-500 bp 500-10000 bp  1-10 bp 10-50 bp   
Type                                                                           
Deletion             12453      180        24           23    16184     2999   
Insertion            13828      197        33           31    17798     3435   
Repeat_contraction       2        6        14           41        9      173   
Repeat_expansion         2        4        16           38       13       75   
Tandem_contraction       0        0         1            3        0        0   
Tandem_expansion         0        0         1            9        0        0   

contig_id                                  000001              ...       \
numbers                                     Count              ...        
Size range         50-500 bp 500-10000 bp 1-10 bp 10-50 bp     ...        
Type                                                           ...        
Deletion                4764        45702    8121      112     ...        
Insertion               5744       135363    8191      103     ...        
Repeat_contraction      2449       161085       1        2     ...        
Repeat_expansion        3557       142318       1        3     ...        
Tandem_contraction       370        13581       0        0     ...        
Tandem_expansion          51        56588       0        0     ...        

contig_id             000160               000211                     \
numbers             Total bp                Count                      
Size range         50-500 bp 500-10000 bp 1-10 bp 10-50 bp 50-500 bp   
Type                                                                   
Deletion                   0         5754       0        0         0   
Insertion                  0            0       0        0         0   
Repeat_contraction         0            0       0        0         0   
Repeat_expansion           0         2857       0        0         0   
Tandem_contraction         0            0       0        0         0   
Tandem_expansion           0            0       0        0         0   

contig_id                                                                 
numbers                         Total bp                                  
Size range         500-10000 bp  1-10 bp 10-50 bp 50-500 bp 500-10000 bp  
Type                                                                      
Deletion                      0        0        0         0            0  
Insertion                     1        0        0         0          589  
Repeat_contraction            0        0        0         0            0  
Repeat_expansion              0        0        0         0            0  
Tandem_contraction            0        0        0         0            0  
Tandem_expansion              0        0        0         0            0  

[6 rows x 504 columns]

In [13]:
Assemblytics_summary_df_10kb.sum(level='numbers', axis=1)['Total bp']/1000

Type                Size range  
Insertion           1-10 bp          148.410
                    10-50 bp          25.529
                    50-500 bp         47.452
                    500-10000 bp    1020.632
Deletion            1-10 bp          128.246
                    10-50 bp          24.853
                    50-500 bp         41.440
                    500-10000 bp     466.141
Tandem_expansion    1-10 bp            0.000
                    10-50 bp           0.000
                    50-500 bp          9.570
                    500-10000 bp     429.436
Tandem_contraction  1-10 bp            0.000
                    10-50 bp           0.000
                    50-500 bp          5.026
                    500-10000 bp     116.641
Repeat_expansion    1-10 bp            0.049
                    10-50 bp           0.827
                    50-500 bp         27.311
                    500-10000 bp    1209.495
Repeat_contraction  1-10 bp            0.068
                    10

In [14]:
#here get the summary of all Types of variations by size intervals given by Assembletics
Size_summary_10kb = Assemblytics_summary_df_10kb.sum(level='numbers', axis=1)['Total bp']/1000
index_a = [x.replace(' bp', '') for x in Size_summary_10kb["Insertion"].index]
var_types = Size_summary_10kb.index.levels[0]



In [15]:
##Determine pwh size
pwh_list = pd.read_csv('/Users/havasquezgross/rust_project/COV/pst130_v5_pwh_ctg.txt',\
sep='\t', header=None)[0].tolist()
length_header = ["contig", "length"]
p_lengths = pd.read_csv('/Users/havasquezgross/rust_project/assembly/pst130_v5_primary.length',\
sep='\t', header=None, names=length_header)
pwh_size = p_lengths[p_lengths['contig'].isin(pwh_list)]['length'].sum()
pwh_size

75861112

In [16]:
#here get the summary of all Types of variations by size intervals given by Assembletics
Size_summary_10kb = Assemblytics_summary_df_10kb.sum(level='numbers', axis=1)['Total bp']/1000
index_a = [x.replace(' bp', '') for x in Size_summary_10kb["Insertion"].index]
var_types = Size_summary_10kb.index.levels[0]


In [17]:
#here get the summary of all Types of variations by size intervals given by Assembletics
Size_summary_50kb = Assemblytics_summary_df_50kb.sum(level='numbers', axis=1)['Total bp']/1000
index_a = [x.replace(' bp', '') for x in Size_summary_50kb["Insertion"].index]
var_types = Size_summary_50kb.index.levels[0]


In [18]:
#Assemblytics_summary_df_10kb
Size_summary_10kb

Type                Size range  
Insertion           1-10 bp          148.410
                    10-50 bp          25.529
                    50-500 bp         47.452
                    500-10000 bp    1020.632
Deletion            1-10 bp          128.246
                    10-50 bp          24.853
                    50-500 bp         41.440
                    500-10000 bp     466.141
Tandem_expansion    1-10 bp            0.000
                    10-50 bp           0.000
                    50-500 bp          9.570
                    500-10000 bp     429.436
Tandem_contraction  1-10 bp            0.000
                    10-50 bp           0.000
                    50-500 bp          5.026
                    500-10000 bp     116.641
Repeat_expansion    1-10 bp            0.049
                    10-50 bp           0.827
                    50-500 bp         27.311
                    500-10000 bp    1209.495
Repeat_contraction  1-10 bp            0.068
                    10

In [19]:
#summarizes here for 10kb

Total_summary_df_10kb = Assemblytics_summary_df_10kb.unstack().sum(level='numbers', axis=1)
Total_summary_df_10kb['variation [1/kbp]'] = Total_summary_df_10kb['Total bp']/pwh_size * 1000
total_relative_var_10kb = float(Total_summary_df_10kb['Total bp'].sum())/pwh_size*100
print(round(total_relative_var_10kb, 2))
Total_summary_df_10kb.to_csv(os.path.join(FIGURE_PATH,  'pst130_v5_struc_variation.tsv'), index = None, sep ='\t')
Total_summary_df_10kb

6.52


numbers,Count,Total bp,variation [1/kbp]
Type,,,
Deletion,100757,660680,8.709073
Insertion,117364,1242023,16.372328
Repeat_contraction,539,1248464,16.457233
Repeat_expansion,505,1237682,16.315105
Tandem_contraction,48,121667,1.603813
Tandem_expansion,133,439006,5.786970


In [20]:
Total_summary_df_10kb['Total bp'].sum()

4949522

In [21]:
#summarizes here for 50kb
Total_summary_df_50kb = Assemblytics_summary_df_50kb.unstack().sum(level='numbers', axis=1)
Total_summary_df_50kb['variation [1/kbp]'] = Total_summary_df_50kb['Total bp']/pwh_size * 1000
total_relative_var_50kb = float(Total_summary_df_50kb['Total bp'].sum())/pwh_size*100
print(round(total_relative_var_50kb, 2))
Total_summary_df_50kb

13.53


numbers,Count,Total bp,variation [1/kbp]
Type,,,
Deletion,100771,864448,11.395140
Insertion,117415,2042396,26.922832
Repeat_contraction,620,2759329,36.373432
Repeat_expansion,610,3269877,43.103468
Tandem_contraction,53,204460,2.695189
Tandem_expansion,168,1123173,14.805649


In [22]:
Total_summary_df_50kb['Total bp'].sum()

10263683